In [1]:
# In case we are using non-OTF data reading methods
# WARNING!!! This is not a fully working code. This is skeletal toy model that needs to be 
# altered based on requirement.

import h5py
import numpy as np

class ReadContiguousData():
    """ Used to augment the start time of noise samples from continuous noise .hdf file """
    # This will become the primary noise reading function
    def __init__(self, data_path="", sample_length=17.0):
        self.sample_length = sample_length
        self.data_path = data_path
        self.dt = None
        self.detectors = None
        # Keep all required files open
        self.data = h5py.File(self.data_path, 'r')
    
    def _make_sample_start_time(self, start_idx, end_idx):
        # Make a sample start time that is uniformly distributed within segdur
        return int(np.random.uniform(low=start_idx, high=end_idx))

    def get_data(self, idx_params):
        sample = []
        for det, idx_param in zip(self.detectors, idx_params):
            key, start_idx, end_idx = idx_param
            sample_start_idx = self._make_sample_start_time(start_idx, end_idx)
            # Get the required portion of given segment
            sidx = sample_start_idx
            eidx = sample_start_idx + int(self.sample_length / self.dt)
            key = f'{det}/{key}' # address of hdf5 dataset required
            # Get time series from segment
            ts = np.array(self.data[key][sidx:eidx]).astype(np.float64)
            sample.append(ts)

        noise = np.stack(noise, axis=0)
        return noise
    
    def get_idx_params(self):
        # Write a function to get start and end idx using segment duration
        # This can be called and saved in __init__ if needed
        pass

    def apply(self):
        idx_params = self.get_idx_params()
        # Get sample with random start time (uniform within segment)
        noise = self.get_noise_segment(idx_params)
        # Return noise data
        return noise

In [2]:
# Using ExternalLinks to access multiple files without opening all of them in an array
import os
import glob
import itertools
import random

def make_elink_lookup(self):
    # Make a lookup table with (id, path, target) for each training data
    # Should have an equal ratio, shuffled, of both classes (signal and noise)
    # Save the dataset paths alongside the target and ids as hdf5
    
    # Other params to be saved in extlink file
    other_params = ['gw_param1', 'gw_param2']
    # Save params
    all_data = {'id': [], 'path': [], 'target': []}
    # Save other params into all_data
    oparams_dict = {foo: [] for foo in other_params}
    all_data.update(oparams_dict)
    
    save_dir = ""
    lookup = os.path.join(save_dir, 'extlinks.hdf')
    main = h5py.File(lookup, 'a', libver='latest')
    
    idx = 0

    # tmp var
    tmp_var = {foo:[] for foo in all_data.keys()}
    # Add attributes if present
    add_attrs = True
    
    dataset_dir = ""
    chunks = os.path.join(dataset_dir, "*.hdf")
    for nfile, chunk_file in enumerate(glob.glob(chunks)):
        ## Read the file, get all data and add data as ExternalLink to extlinks.hdf file
        # Read HDF5 tree and add tree to ExternalLink
        with h5py.File(chunk_file, 'r') as h5f:
            # fp.keys() --> Groups in HDF5 file
            # fp[group_key].keys() --> Further groups present within
            grps = list(h5f.keys())
            
            for grp in grps:
                # Get sample length of each dataset
                # This will make a branch with chunk file number in the address
                # Like: 007/grp (007 being the file number)
                _branch = f"{nfile:03}" + '/' + grp
                
                for obj in h5f[grp].keys():
                    # Add dataset as External Link
                    # Branch Format: 007 / group / sub_group / dataset
                    curr_branch = grp + '/' + obj # branch in chunk file to refer
                    extl_branch = _branch + '/' + obj # branch details for external link
                    # Make external link
                    main[extl_branch] = h5py.ExternalLink(chunk_file, curr_branch)
                
                # Maxshape of given np.ndarray
                shape = np.array(h5f[grp][list(h5f[grp].keys())[0]]).shape[0]
                # Other params of each sample
                for oparam in other_params:
                    # Assuming other datasets present (like gw params)
                    if oparam in h5f[grp].keys():
                        oparam_ = np.array(h5f[grp][oparam])
                    else:
                        # Noise files may not have these params
                        # So we can save a bunch of -1's in them for constant maxshape
                        oparam_ = np.full(shape, -1)
                    # Update oparam
                    tmp_var[oparam].extend(oparam_)

                # Update idxs (unique ids for each sample)
                branch_ids = np.arange(idx, idx+shape)
                tmp_var['id'].extend(branch_ids)
                idx = idx+shape
                # Update paths to dataset objects (address to each unique sample)
                # We external linked entire array to extlinks.hdf
                # We need to give unique addresses to each sample in that nd-array
                # Here shape will be the number of samples we have in said array
                branch_paths = itertools.product([_branch], np.arange(shape))
                tmp_var['path'].extend([foo + '/' + str(bar) for foo, bar in branch_paths])
                # Update target variable for each sample
                target = None
                tmp_var['target'].extend(np.full(shape, target))
                    
            # Add attributes from chunk files once to main ExternalLink File
            if add_attrs:
                attrs_save = dict(h5f.attrs)
                for key, value in attrs_save.items():
                    main.attrs[key] = value
                add_attrs = False

    # Sanity check
    # Uncomment this to check whether external links worked properly
    """ 
    with h5py.File(self.dirs['lookup'], 'a', libver='latest') as fp:
        start = time.time()
        # Change address to known address
        # This is an example from the ORChiD pipeline
        print(np.array(fp['004/2048/signal/h_plus'][2]))
        fin = time.time() - start
        print(fin)
    raise
    """
    
    ## Create lookup using zip (use column_stack instead)
    # Explicitly check for length inconsistancies. zip doesn't raise error.
    tmp_var_lens = [len(tmp_var[foo]) for foo in tmp_var.keys()]
    # Checks if all lengths are the same (rewrite this in a better way, using numpy probably)
    assert len(list(set(tmp_var_lens))) == 1, 'var:tmp_var fields in extlinks have inconsistent column lengths!'
    
    lookup = list(zip(*tmp_var.values()))
    # Shuffle the column stack (signal and noise are not shuffled)
    random.shuffle(lookup)
    # Separate out the tuples for ids, paths and targets
    for foo, key in zip(zip(*lookup), tmp_var.keys()):
        all_data[key].extend(foo)
    
    # Close file explicitly, or use with instead
    main.close()
    
    # Write required fields as datasets in HDF5 extlinks.hdf file
    with h5py.File(lookup, 'a') as ds:
        """
        Shuffle Filter for HDF5:
            Block-oriented compressors like GZIP or LZF work better when presented with 
            runs of similar values. Enabling the shuffle filter rearranges the bytes in 
            the chunk and may improve compression ratio. No significant speed penalty, 
            lossless.
        """
        for _param in all_data.keys():
            ds.create_dataset(_param, data=all_data[_param], compression='gzip', compression_opts=9, shuffle=True)
        
    print("make_elink_lookup: ExternalLink lookup table created successfully!")


Once the external links are created, we can access them using the unique addresses stored in the extlinks file.

In [3]:
def read_extlinks():
    # Set lookup table ('training.hdf' lookup table is also present for batch-loading)
    lookup_file = 'extlinks.hdf'

    export_dir = ""
    lookup_table = os.path.join(export_dir, lookup_file)
    # Deprecated: Usage of self.get_metadata with make_default_dataset (non-MP)
    with h5py.File(lookup_table, 'a') as fp:
        ids = np.array(fp['id'][:])
        # For reading strings
        paths = np.array([foo.decode('utf-8') for foo in fp['path']])
        targets = np.array(fp['target'][:])

In [ ]:
# Here data_path is a single instance of paths (addresses to unique samples)
# In a class, open the extliks file as self.extmain
# We can then use the addresses to access the linked samples

def read(self, data_path):
    # Store all params within chunk file
    params = {}
    targets = {}
    
    # Get data from ExternalLink'ed lookup file
    # The addresses contain dataset address and sample idx
    # We split them to we access the address normally and access the required idx
    HDF5_Dataset, didx = os.path.split(data_path)
    # Dataset Index should be an integer
    didx = int(didx)
    # Access group
    group = self.extmain[HDF5_Dataset]
    
    # Targets can be stored in the extlinks file
    target = None
    targets['gw'] = target

    if not target:
        ## Read noise data (for two detectors, each stored as separate dataset)
        noise_1 = np.array(group['noise_1'][didx])
        noise_2 = np.array(group['noise_2'][didx])
        sample = np.stack([noise_1, noise_2], axis=0)
    else:
        ## Read signal data
        h_plus = np.array(group['h_plus'][didx])
        h_cross = np.array(group['h_cross'][didx])
        sample = np.stack([h_plus, h_cross], axis=0)
        # Signal params
        params['mass1'] = group['mass1'][didx]
        params['mass2'] = group['mass2'][didx]
        params['distance'] = group['distance'][didx]
        params['mchirp'] = group['mchirp'][didx]
        params['tc'] = group['tc'][didx]
        params['ra'] = group['ra'][didx]
        params['dec'] = group['dec'][didx]
        params['polarization'] = group['polarization'][didx]
    
    # Generic params
    params['sample_rate'] = self.sample_rate
    params['noise_low_freq_cutoff'] = self.noise_low_freq_cutoff
    
    return (sample, targets, params)